# Uchazeč: data o uchazečích o studium na pedagogických fakultách

- přejmenuj sloupce v datasetu, aby to mělo logičtější strukturu
- připrav variable_labels a value_labels pro export do staty

## Importy

In [1]:
# nejake standardni importy
import os
import sys
import pyreadstat
import pandas as pd
import numpy as np
import re
from statsmodels.stats.weightstats import DescrStatsW
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
# aby grafy byly rozumně čitelné na obrazovce
plt.rcParams['figure.dpi'] = 90
plt.rcParams['axes.titlesize'] = 11
plt.rcParams['figure.figsize'] = 10, 5
#plt.ioff()
import dbf

In [2]:
data_root = '/mnt/d/projects/idea/data'
path17 = 'uchazec/0022MUCH17P'
path21 = 'uchazec/0022MUCH21P'

In [3]:
def loader(year=21):
    
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250', low_memory=False)
    print('1 ', df.shape)
    
    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS', 'VYPR', 'ZAPS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    to_rename = {
        'RDAT': 'dat_sber',
        'RID': 'fak_id',
        'ROD_KOD': 'id',
        'STATB': 'bydliste_stat',
        'OBECB': 'bydliste_obec',
        'PSCB': 'bydliste_psc',
        'STAT': 'stat',
        'ODHL': 'odhl',
        'IZOS': 'ss_izo',
        'OBSS': 'ss_obor',
        'RMAT': 'rmat',
        'TYP_ST': 'typ_st',
        'FORMA_ST': 'forma_st',
        'PROGRAM': 'program',
        'OBOR1': 'obor1',
        'OBOR2': 'obor2',
        'OBOR3': 'obor3',
        'OBOR4': 'obor4',
        'OBOR5': 'obor5',
        'APRO1': 'apro1',
        'APRO2': 'apro2',
        'APRO3': 'apro3',
        'DAT_REG': 'dat_reg',
        'VYPR': 'vypr',
        'DAT_VYPR': 'dat_vypr',
        'ZAPS': 'zaps',
        'DAT_ZAPS': 'dat_zaps'
    }

    to_drop = ['CHYV']
    
    value_labels = {}
    variable_labels = {}
    
    df = df.rename(columns=to_rename).drop(columns=to_drop)
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['IDX'] = aast.index
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df.rename(columns={'stat': 'KOD'}), aast[['KOD', 'IDX']].rename(columns={'IDX': 'stat'}), 
                  how='left').drop(columns=['KOD'])
    df['stat_iso'] = df['stat']
    df = pd.merge(df.rename(columns={'bydliste_stat': 'KOD'}), aast[['KOD', 'IDX']].rename(columns={'IDX': 'bydliste_stat'}), 
                  how='left').drop(columns=['KOD'])
    df['bydliste_stat_iso'] = df['bydliste_stat']
    
    aast_dict = aast[['IDX', 'ZKR']].set_index('IDX')['ZKR'].to_dict()
    aast_iso_dict = aast[['IDX', 'ISO_ALPHA3']].set_index('IDX')['ISO_ALPHA3'].to_dict()
    value_labels['stat'] = aast_dict
    value_labels['stat_iso'] = aast_iso_dict
    value_labels['bydliste_stat'] = aast_dict
    value_labels['bydliste_stat_iso'] = aast_iso_dict
    
    # registers for ODHL, TYP_ST, FORMA_ST, VYPR, ZAPS
    registers = {
        'odhl': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'typ_st': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'forma_st' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'vypr': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'zaps': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    print('2 ', df.shape)
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        rg['IDX'] = rg.index
        df = pd.merge(df.rename(columns={c: 'KOD'}), rg[['KOD', 'IDX']].rename(columns={'IDX': c}), 
                      how='left').drop(columns=['KOD'])        
        rg_dict = rg[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()
        value_labels[c] = rg_dict
    
    # gender etc.
    value_labels['gender'] = {0: 'Dívka', 1: 'Chlapec', 2: 'Neznámé'}
    df['je_rc'] = df['id'].str[4:6] == 'QQ'
    df['rc1'] = np.where(df['je_rc'], pd.to_numeric(df['id'].str[:2], errors='coerce'), np.nan)
    df['rc2'] = np.where(df['je_rc'], pd.to_numeric(df['id'].str[2:4], errors='coerce'), np.nan)
    df['gender'] = np.where(df['je_rc'], np.where(df['rc2'] > 50, 0, 1), 2)
    df['nar_rok'] = np.where(df['je_rc'], np.where(df['rc1'] < year - 5, 2000 + df['rc1'], 1900 + df['rc1']), np.nan)
    df['nar_mesic'] = np.where(df['je_rc'], np.where(df['rc2'] > 50, df['rc2'] - 50, df['rc2']), np.nan)
    df = df.drop(columns=['je_rc', 'rc1', 'rc2'])
    df['vek'] = 2000 + year - df['nar_rok'] + (9 - df['nar_mesic']) / 12
    
    print('3 ', df.shape)    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    def selector(frame):
        # just use the longest one - the shorter ones typically use abbreviations
        frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
        return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

    ss['OBOR'] = ss['OBOR'].str.strip()
#     obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates().groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
#     obory['IDX'] = obory.index
    
#     df = pd.merge(df.rename(columns={'ss_obor': 'OBOR'}), obory[['OBOR', 'IDX']].rename(columns={'IDX': 'ss_obor'}),
#                   how='left').drop(columns=['OBOR'])
#     value_labels['ss_obor'] = obory[['IDX', 0]].set_index('IDX')[0].to_dict()
    
    #sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates().reset_index(drop=True)    
    sss = ss[['IZO', 'VUSC']].drop_duplicates().reset_index(drop=True)    
    # SS is complicated -> too lengthy for stata value labels...
    # df['ss'] = df['ss_izo']
    df = pd.merge(df, sss[['IZO', 'VUSC']].rename(columns={'IZO': 'ss_izo', 'VUSC': 'ss_nuts'}), how='left')
    # df['ss_nuts'] = df['ss_izo']
    # value_labels['ss'] = sss[['IZO', 'ZAR_PLN']].set_index('IZO')['ZAR_PLN'].to_dict()
    # value_labels['ss_nuts'] = sss[['IZO', 'VUSC']].set_index('IZO')['VUSC'].to_dict()

    print('4 ', df.shape)
    
    odhl_ss = {v: k for k, v in value_labels['odhl'].items()}['Střední škola']
    value_labels['ss_typ'] = {9: 'Není SŠ', 0: 'SOŠ', 1: 'Gymnázium', 2: 'Jiné'}
    df['ss_typ'] = np.where(df['odhl'] != odhl_ss, 9, np.where(df['ss_obor'] == '', 2,
                                                               np.where(df['ss_obor'].str.startswith('794'), 1, 0)))
    df['ss_gym_delka'] = np.where(df['ss_typ'] == 1, pd.to_numeric(df['ss_obor'].str[5], errors='coerce'), np.nan)

    print('5 ', df.shape)
    
    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro = regpro[['RID', 'ZAR_PLN', 'VS_PLN', 'UZEMI']]
    regpro['IDX'] = regpro.index
    regpro['FAK_S_VS'] = regpro['VS_PLN'] + ', ' + regpro['ZAR_PLN']
    
    vs = regpro[['VS_PLN']].drop_duplicates().sort_values('VS_PLN').reset_index(drop=True)
    vs['VS_IDX'] = vs.index
    
    regpro = pd.merge(regpro, vs)
    df = pd.merge(df, regpro[['IDX', 'RID', 'VS_IDX', 'UZEMI']]
                  .rename(columns={'IDX': 'fak_nazev', 'VS_IDX': 'vs_nazev', 'RID': 'fak_id', 'UZEMI': 'fak_nuts'}),
                  how='left')
    
    print('6 ', df.shape)
    
    # df['fak_plny'] = df['fak_nazev']
    value_labels['fak_nazev'] = regpro[['IDX', 'ZAR_PLN']].set_index('IDX')['ZAR_PLN'].to_dict()
    # value_labels['fak_plny'] = regpro[['IDX', 'FAK_S_VS']].set_index('IDX')['FAK_S_VS'].to_dict()
    value_labels['vs_nazev'] = vs[['VS_IDX', 'VS_PLN']].set_index('VS_IDX')['VS_PLN'].to_dict()    
    
    # this is sensible only for 2017 data...
#     akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
#     akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
#     for i in [1, 2, 5]:
#         df[f'obor1_{i}'] = df['obor1'].str[:i]
    
#     akko1 = akko[akko['KOD'].str.len() == 1].copy()
#     akko1['KOD'] = pd.to_numeric(akko1['KOD'])
#     value_labels['obor1_1'] = akko1[['KOD', 'TXT']].set_index('KOD')['TXT'].to_dict()
    
#     akko2 = akko[akko['KOD'].str.len() == 2].copy()
#     akko2['KOD'] = pd.to_numeric(akko2['KOD'])
#     value_labels['obor1_2'] = akko2[['KOD', 'TXT']].set_index('KOD')['TXT'].to_dict()
    
#     akko5 = akko[akko['KOD'].str.len() == 5].copy().reset_index(drop=True)
#     akko5['IDX'] = akko5.index

#     df = pd.merge(df.rename(columns={'obor1_5': 'KOD'}), akko5[['KOD', 'IDX']].rename(columns={'IDX': 'obor1_5'}), 
#                   how='left').drop(columns=['KOD'])
#     value_labels['obor1_5'] = akko5[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()    
    
    # program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    # program['IDX'] = program.index
    # df = pd.merge(df.rename(columns={'program': 'KOD'}), program[['KOD', 'IDX']].rename(columns={'IDX': 'program'}), 
    #               how='left').drop(columns=['KOD'])    
    # value_labels['program'] = program[['IDX', 'NAZEV']].set_index('IDX')['NAZEV'].to_dict()
    
#     akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
#     akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
#     akvo['IDX'] = akvo.index
#     akvo_dict = akvo[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()
        
#     for i in range(1, 6):
#         df = pd.merge(df.rename(columns={f'obor{i}': 'KOD'}), akvo[['KOD', 'IDX']].rename(columns={'IDX': f'obor{i}'}), 
#                       how='left').drop(columns=['KOD'])        
#         value_labels[f'obor{i}'] = akvo_dict
    
    variable_labels = {
        'id': 'Identifikátor uchazeče (kódované rodné číslo)',
        'gender': 'Pohlaví uchazeče',
        'nar_rok': 'Rok narození',
        'nar_mesic': 'Měsíc narození',
        'vek': 'Přibližný věk uchazeče',
        'stat': 'Státní příslušnost uchazeče',
        'stat_iso': 'Státní příslušnost uchazeče (ISO)',
        'bydliste_stat': 'Stát trvalého pobytu',
        'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
        'bydliste_obec': 'Obec trvalého pobytu',
        'bydliste_psc': 'PSČ trvalého pobytu',
        'fak_id': 'Identifikátor fakulty',
        'fak_nazev': 'Název fakulty',
        # 'fak_plny': 'Název fakulty včetně VŠ',
        'fak_nuts': 'Lokalita fakulty',
        'vs_nazev': 'Název vysoké školy',
        'odhl': 'Odkud se uchazeč hlásí',
        'ss_izo': 'Identifikátor střední školy',
        # 'ss': 'Střední škola',
        'ss_nuts': 'NUTS region střední školy',
        'ss_obor': 'Obor studia střední školy',
        'ss_typ': 'Typ střední školy',
        'ss_gym_delka': 'Délka studia gymnázia',
        'rmat': 'Rok maturity',
        'typ_st': 'Typ studia',
        'forma_st': 'Forma studia',
        'vypr': 'Výsledek přijímacího řízení',
        'zaps': 'Výsledek zápisu',
        'program': 'Studijní program',
        'obor1': 'Studijní obor',
        'obor2': 'Studijní obor',
        'obor3': 'Studijní obor',
        'obor4': 'Studijní obor',
        'obor5': 'Studijní obor',
        'apro1': 'Aprobace',
        'apro2': 'Aprobace',
        'apro3': 'Aprobace',
        # 'obor1_1': 'Skupina programů podle prvního oboru',
        # 'obor1_2': 'Podskupina programů podle prvního oboru',
        # 'obor1_5': 'Program podle prvního oboru',
        'dat_sber': 'Rozhodné datum sběru',
        'dat_reg': 'Datum podání (registrace) přihlášky',
        'dat_vypr': 'Datum rozhodnutí o výsledku přijímacího řízení',
        'dat_zaps': 'Datum zápisu',
    }

    df = df[variable_labels.keys()]
    
    return df, variable_labels, value_labels
    

In [18]:
li = []
for y in range(1999, 2023):
    pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
    pdf['year'] = y
    li.append(pdf)
ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
ss = ss.sort_values('year', ascending=False).drop_duplicates('IZO').sort_index()
ss['OBOR'] = ss['OBOR'].str.strip()

sss = ss[['IZO', 'VUSC']].drop_duplicates().reset_index(drop=True)    


In [19]:
sss.shape

(2123, 2)

In [20]:
len(sss['IZO'].unique())

2123

In [21]:
sss['IZO'].value_counts()

69230        1
44012667     1
45235686     1
45162654     1
45019517     1
            ..
47274697     1
46414991     1
44808208     1
44283644     1
150004648    1
Name: IZO, Length: 2123, dtype: int64

In [8]:
sss[sss['IZO'] == 10622110]

,IZO,VUSC
1667,10622110,CZ0102
1747,10622110,CZ0101
2090,10622110,CZ0108
2134,10622110,CZ0106


In [16]:
ss[ss['IZO'] == 10622110]

,IZO,ZAR_PLN,MISTO,ULICE,CP,PSC,VUSC,OBOR,OBOR_NAZ,PLN_NAZ,year,RED_IZO,ZAR_ZKR,POM
5608,10622110,Soukromá taneční konzervatoř ...,Praha 2,Na Smetance 1,505,12000,CZ0102,8246N001,Tanec,Tanec ...,2001,NaN,NaN,NaN
7983,10622110,Soukromá taneční konzervatoř ...,Praha 2,Na Smetance 1,505,12000,CZ0102,8246N001,Tanec,Tanec ...,2002,NaN,NaN,NaN
10590,10622110,1. soukromá taneční konzervatoř v Praze ...,Praha 1,Michalská 432,12,11000,CZ0101,8246N001,Tanec,Tanec ...,2003,NaN,NaN,NaN
16243,10622110,1. soukromá taneční konzervatoř v Praze ...,Praha 1,Michalská 432,12,11000,CZ0101,8246N001,Tanec,Tanec ...,2005,NaN,NaN,NaN
19203,10622110,1. soukr. taneční konzervatoř ...,Praha 1,Michalská 432,12,11000,CZ0101,8246N001,Tanec,Tanec ...,2006,NaN,NaN,NaN
22242,10622110,1. soukr. taneční konzervatoř ...,Praha 1,Michalská 432,12,11000,CZ0101,8246N001,Tanec,Tanec ...,2007,NaN,NaN,NaN
37576,10622110,"Taneční konzervatoř Ivo Váni - Psoty v Praze, ...",Praha 8,Thámova 221,7,18600,CZ0108,8246M001,Tanec,Tanec ...,2011,600004805.0,"Tan.konzervatoř I.V-Psoty v Praze,s.r.o.",NaN
37589,10622110,"Taneční konzervatoř Ivo Váni - Psoty v Praze, ...",Praha 8,Thámova 221,7,18600,CZ0108,8246N001,Tanec,Tanec ...,2011,600004805.0,"Tan.konzervatoř I.V-Psoty v Praze,s.r.o.",NaN
38904,10622110,"Taneční konzervatoř Ivo Váni - Psoty v Praze, ...",Praha 8,Thámova 221,7,18600,CZ0108,8246M001,Tanec,Tanec ...,2012,600004805.0,"Tan.konzervatoř I.V-Psoty v Praze,s.r.o.",NaN
49340,10622110,Pražská taneční konzervatoř a střední odborná ...,Praha 6 - Řepy,Laudova 1024/10,,16300,CZ0106,8246M001,Tanec,Tanec ...,2014,600005666.0,Pražská taneční konzervatoř a SOŠ s.r.o.,NaN


In [ ]:
ss.sort_values(year, ascending=False).drop_duplicates('IZO').sort_index()

In [ ]:
# 'OBSS': 
for c, url in registers.items():
rg = pd.read_xml('http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml', encoding='cp1250', xpath='./veta')
rg['IDX'] = rg.index

    df = pd.merge(df.rename(columns={c: 'KOD'}), rg[['KOD', 'IDX']].rename(columns={'IDX': c}), 
                  how='left').drop(columns=['KOD'])        
    rg_dict = rg[['IDX', 'TXT']].set_index('IDX')['TXT'].to_dict()
    value_labels[c] = rg_dict

In [12]:
rg = pd.read_xml('http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml', encoding='cp1250', xpath='./veta')
rg['IDX'] = rg.index

In [13]:
rg

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN,IDX
0,AKSO,1601M001,Ochr.a tvorba živ.prostř.,Ochrana a tvorba životního prostředí,1.9.1998,31.12.2999,1601600R08,None,0
1,AKSO,1601M002,Ochr.přírody a prostředí,Ochrana přírody a prostředí,1.9.1998,31.12.2999,1612600R08,None,1
2,AKSO,1601M004,Ekol.a ochrana krajiny,Ekologie a ochrana krajiny,1.9.1998,31.12.2999,1622600R08,None,2
3,AKSO,1601M005,Ochr.a obnova živ.prostř.,Ochrana a obnova životního prostředí,1.9.1998,31.12.2999,1629600R08,None,3
4,AKSO,1601M01,Ekologie a životní prost.,Ekologie a životní prostředí,31.8.2008,31.12.2999,*******,RVP,4
...,...,...,...,...,...,...,...,...,...
1594,AKSO,8251L06,Um.-řem.stavba hud.nástr.,Uměleckořemeslná stavba hudebních nástrojů,31.8.2009,31.12.2999,*******,None,1594
1595,AKSO,8251L501,Umělecké řemeslné práce,Umělecké řemeslné práce,1.9.1998,31.12.2999,8501400R09,vyř. k 1.9.2012,1595
1596,AKSO,8251L51,Umělecké řemeslné práce,Umělecké řemeslné práce,31.8.2009,31.12.2999,*******,None,1596
1597,AKSO,8251L536,Scénická tvorba,Scénická tvorba,10.12.1999,31.12.2999,8502400R09,vyř. k 1.9.2012,1597


In [14]:
rg[rg['KOD'] == 10622110]

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN,IDX


In [142]:
df, variable_labels, value_labels = loader()

1  (284201, 28)
2  (284201, 29)
3  (284201, 33)
4  (287853, 34)
5  (287853, 36)
6  (287853, 39)


In [128]:
variable_labels

{'id': 'Identifikátor uchazeče (kódované rodné číslo)',
 'gender': 'Pohlaví uchazeče',
 'nar_rok': 'Rok narození',
 'nar_mesic': 'Měsíc narození',
 'vek': 'Přibližný věk uchazeče',
 'stat': 'Státní příslušnost uchazeče',
 'stat_iso': 'Státní příslušnost uchazeče (ISO)',
 'bydliste_stat': 'Stát trvalého pobytu',
 'bydliste_stat_iso': 'Stát trvalého pobytu (ISO)',
 'bydliste_obec': 'Obec trvalého pobytu',
 'bydliste_psc': 'PSČ trvalého pobytu',
 'fak_id': 'Identifikátor fakulty',
 'fak_nazev': 'Název fakulty',
 'fak_nuts': 'Lokalita fakulty',
 'vs_nazev': 'Název vysoké školy',
 'odhl': 'Odkud se uchazeč hlásí',
 'ss_izo': 'Identifikátor střední školy',
 'ss_nuts': 'NUTS region střední školy',
 'ss_obor': 'Obor studia střední školy',
 'ss_typ': 'Typ střední školy',
 'ss_gym_delka': 'Délka studia gymnázia',
 'rmat': 'Rok maturity',
 'typ_st': 'Typ studia',
 'forma_st': 'Forma studia',
 'vypr': 'Výsledek přijímacího řízení',
 'zaps': 'Výsledek zápisu',
 'program': 'Studijní program',
 'obor

In [131]:
for c in df.columns:
    if c in value_labels.keys():
        df[c] = df[c].map(value_labels[c])

In [134]:
df.head(n=1000).to_parquet('temp/uchazec/uch21.parquet')

## Filtrování dat

- QQ v id, pak zkontroluj kolik je opakovaných záznamů; tím bych měl mít také jenom českou národnost
- Hlásí se ze střední školy

Kolik žáků tím vyřadím?

In [135]:
# QQ v id
(df['id'].str[4:6] == 'QQ').value_counts()

True     782957
False     45718
Name: id, dtype: int64

In [136]:
df.shape

(828675, 39)

In [137]:
raw = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250', low_memory=False)

In [138]:
raw.shape

(284201, 28)

In [119]:
df, variable_labels, value_labels = loader(year=17)
df.to_stata(f'{data_root}/uchazec/uch17.dta', write_index=False, version=118, variable_labels=variable_labels, value_labels=value_labels)

/tmp/ipykernel_401/221791032.py:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')


In [56]:
akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
akvo.head()

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AKVO,1101R006,Deskriptivní geometrie,Deskriptivní geometrie,1.1.2003,31.12.2999,None,None
1,AKVO,1101R008,Diskrétní matematika,Diskrétní matematika,1.1.2003,31.12.2999,None,None
2,AKVO,1101R014,Matematická analýza,Matematická analýza,1.1.2003,31.12.2999,11047,None
3,AKVO,1101R016,Matematika,Matematika,1.1.2003,31.12.2999,11617,None
4,AKVO,1101R018,"Matem.,aplikace(analýza)",Matematika a její aplikace (zaměření matematic...,1.1.2003,31.12.2999,None,None


In [66]:
# načti data z roku 2021
df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')

/tmp/ipykernel_401/367580044.py:2: DtypeWarning: Columns (3,6,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_root}/{path21}.csv', encoding='cp1250')


In [67]:
df[:1000].to_csv('temp/uchazec21.csv')

In [79]:
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.to_csv('temp/uchazec/program.csv')

In [80]:
akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
akvo.to_csv('temp/uchazec/akvo.csv')

In [84]:
akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
akko.to_csv('temp/uchazec/akko2.csv')

In [85]:
regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
regpro.to_csv('temp/uchazec/regpro.csv')

In [88]:
pd.to_numeric(df['fak_id'])

ValueError: Unable to parse string "6N900" at position 20

In [109]:
regpro['VS_PLN'] + '___' + regpro['ZAR_PLN']

0      Jihočeská univerzita v Českých Budějovicích   ...
1      Slezská univerzita v Opavě                    ...
2      Česká zemědělská univerzita v Praze           ...
3      Univerzita Palackého v Olomouci               ...
4      Západočeská univerzita v Plzni                ...
                             ...                        
205    Vysoká škola podnikání a práva, a.s.          ...
206    Vysoká škola kreativní komunikace             ...
207    Vysoká škola finanční a správní, a.s.         ...
208    Technická univerzita v Liberci                ...
209    Vysoká škola ekonomie a managementu, a.s.     ...
Length: 210, dtype: object

In [68]:
program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
program.head()

,KOD,NAZEV,RID
0,B0111A190001,Pedagogika ...,14210
1,B0111A190002,Sociální pedagogika a volný čas ...,14410
2,B0111A190003,Andragogika ...,15210
3,B0111A190004,Andragogika a personální řízení ...,11210
4,B0111A190005,Sociální pedagogika a poradenství ...,14210


In [65]:
df['program']

0         M0421A220001
1         B0321A180002
2                M5103
3         M0421A220001
4         B0413P050025
              ...     
284196    B0411P050002
284197    B0111A190023
284198    B0923A240005
284199    B0923A240005
284200           B6739
Name: program, Length: 284201, dtype: object

In [60]:
value_labels['obor1_1']

{1: 'PŘÍRODNÍ VĚDY A NAUKY',
 2: 'TECHNICKÉ VĚDY A NAUKY (1. část)',
 3: 'TECHNICKÉ VĚDY A NAUKY (2. část)',
 4: 'ZEMĚDĚLSKO-LESNICKÉ A VETERINÁRNÍ VĚDY A NAUKY',
 5: 'ZDRAVOTNICTVÍ, LÉKAŘSKÉ A FARMACEUTICKÉ VĚDY A NAUKY',
 6: 'SPOLEČENSKÉ VĚDY, NAUKY A SLUŽBY (1. část)',
 7: 'SPOLEČENSKÉ VĚDY, NAUKY A SLUŽBY (2. část)',
 8: 'VĚDY A NAUKY O KULTUŘE A UMĚNÍ',
 9: 'VOJENSKÉ VĚDY A NAUKY'}

In [62]:
df['obor1_1'].iloc[1]

''

In [42]:
w = pd.io.stata.StataWriterUTF8(f'{data_root}/uchazec/uch21.dta', df, write_index=False, version=118,
                                variable_labels=variable_labels, value_labels=value_labels)
w.write_file()

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0161' in position 11: ordinal not in range(256)

In [43]:
df.to_stata(f'{data_root}/uchazec/uch21.dta', write_index=False, version=118, variable_labels=variable_labels) #, value_labels=value_labels)

In [ ]:
w = pd.io.stata.StataWriterUTF8('foo.dta', df_master)
w.write_file()

In [16]:
df.head()

,RDAT,RID,ROD_KOD,STATB,OBECB,PSCB,STAT,ODHL,IZOS,OBSS,...,OBOR5,APRO1,APRO2,APRO3,DAT_REG,VYPR,DAT_VYPR,ZAPS,DAT_ZAPS,CHYV
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,NaN,NaN,NaN,23062021,15,23062021,1,23062021,1 00 0 0 000000 0000000
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,NaN,NaN,NaN,28072021,15,28072021,1,28072021,1 00 0 0 000000 0000000
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,NaN,NaN,NaN,8072021,15,08072021,1,08072021,1 00 0 0 000000 0000000
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,NaN,NaN,NaN,10112020,20,10112020,0,,00 01 1 0000 0000 0 0


In [44]:
pd.__version__

'1.4.3'

In [17]:
aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')


In [18]:
aast

,IDC,KOD,ZKR,TXT,DATZP,DATKP,SPC,POZN
0,AAST,0,Zatím nezjištěno,Zatím nezjištěno,1.1.2000,31.12.2999,***************,pouze pro prezentaci
1,AAST,4,Afghánistán,Afghánská islámská republika,1.7.2014,31.12.2999,AFAFG601 5166020,None
2,AAST,8,Albánie,Albánská republika,5.5.2005,31.12.2999,ALALB111 4107015,None
3,AAST,10,Antarktida,Antarktida,5.5.2005,31.12.2999,AQATA961 5089150,None
4,AAST,12,Alžírsko,Alžírská demokratická a lidová republika,1.1.2011,31.12.2999,DZDZA701 5120830,None
...,...,...,...,...,...,...,...,...
247,AAST,876,Wallis a Futuna,Teritorium Wallisovy ostrovy a Futuna,1.7.2014,31.12.2999,WFWLF9933 5081150,None
248,AAST,882,Samoa,Nezávislý stát Samoa,5.5.2005,31.12.2999,WSWSM931 5081950,None
249,AAST,887,Jemen,Jemenská republika,5.5.2005,31.12.2999,YEYEM623 5165320,None
250,AAST,894,Zambie,Zambijská republika,5.5.2005,31.12.2999,ZMZMB785 5137830,None


In [21]:
aast[['KOD', 'ZKR']].set_index('KOD')['ZKR'].to_dict()

{0: 'Zatím nezjištěno',
 4: 'Afghánistán',
 8: 'Albánie',
 10: 'Antarktida',
 12: 'Alžírsko',
 16: 'Americká Samoa',
 20: 'Andorra',
 24: 'Angola',
 28: 'Antigua a Barbuda',
 31: 'Ázerbájdžán',
 32: 'Argentina',
 36: 'Austrálie',
 40: 'Rakousko',
 44: 'Bahamy',
 48: 'Bahrajn',
 50: 'Bangladéš',
 51: 'Arménie',
 52: 'Barbados',
 56: 'Belgie',
 60: 'Bermudy',
 64: 'Bhútán',
 68: 'Bolívie',
 70: 'Bosna a Hercegovina',
 72: 'Botswana',
 74: 'Bouvetův ostrov',
 76: 'Brazílie',
 84: 'Belize',
 86: 'Britské indickooc.území',
 90: 'Šalomounovy ostrovy',
 92: 'Britské Panenské ostr.',
 95: 'Kosovo',
 96: 'Brunej',
 100: 'Bulharsko',
 104: 'Myanmar',
 108: 'Burundi',
 112: 'Bělorusko',
 116: 'Kambodža',
 120: 'Kamerun',
 124: 'Kanada',
 132: 'Kapverdy',
 136: 'Kajmanské ostrovy',
 140: 'Středoafrická republika',
 144: 'Šrí Lanka',
 148: 'Čad',
 152: 'Chile',
 156: 'Čína',
 158: 'Tchaj-wan',
 162: 'Vánoční ostrov',
 166: 'Kokosové (Keeling.) ostr.',
 170: 'Kolumbie',
 174: 'Komory',
 175: 'Mayott

In [7]:
to_rename = {
    'RDAT': 'dat_sber',
    'RID': 'fak_id',
    'ROD_KOD': 'id',
    'STATB': 'bydliste_stat',
    'OBECB': 'bydliste_obec',
    'PSCB': 'bydliste_psc',
    'STAT': 'stat',
    'ODHL': 'odhl',
    'IZOS': 'ss_izo',
    'OBSS': 'ss_obor',
    'RMAT': 'rmat',
    'TYP_ST': 'typ_st',
    'FORMA_ST': 'forma_st',
    'PROGRAM': 'program',
    'OBOR1': 'obor1',
    'OBOR2': 'obor2',
    'OBOR3': 'obor3',
    'OBOR4': 'obor4',
    'OBOR5': 'obor5',
    'APRO1': 'apro1',
    'APRO2': 'apro2',
    'APRO3': 'apro3',
    'DAT_REG': 'dat_reg',
    'VYPR': 'vypr',
    'DAT_VYPR': 'dat_vypr',
    'ZAPS': 'zaps',
    'DAT_ZAPS': 'dat_zaps'
}

to_drop = ['CHYV']

In [12]:
# conversion to numeric
for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# strip white spaces
for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
    df[c] = df[c].str.strip()

df = df.rename(columns=to_rename).drop(columns=to_drop)

In [13]:
df.head()

,dat_sber,fak_id,id,bydliste_stat,bydliste_obec,bydliste_psc,stat,odhl,ss_izo,ss_obor,...,obor4,obor5,apro1,apro2,apro3,dat_reg,vypr,dat_vypr,zaps,dat_zaps
0,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
1,20211031,1423,,703.0,,83107,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,23062021,15,23062021,1,23062021
2,20211031,1411,,703.0,,91624,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,28072021,15,28072021,1,28072021
3,20211031,1422,,703.0,,91108,703.0,2,999999703.0,,...,,,NaN,NaN,NaN,8072021,15,08072021,1,08072021
4,20211031,78900,,566.0,,,566.0,2,999999566.0,,...,,,NaN,NaN,NaN,10112020,20,10112020,0,


## Loader

In [303]:
def loader(year=17):
    # year can be 17 or 21
    data_root = '/mnt/d/projects/idea/data'
    path = f'uchazec/0022MUCH{year}P'
    
    df = pd.read_csv(f'{data_root}/{path}.csv', encoding='cp1250')

    # conversion to numeric
    for c in ['RMAT', 'STAT', 'STATB', 'IZOS'] + [f'APRO{i}' for i in range(1, 4)]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        
    # strip white spaces
    for c in ['OBSS', 'RID', 'PROGRAM'] + [f'OBOR{i}' for i in range(1, 6)]:
        df[c] = df[c].str.strip()
    
    # label STAT, STATB: register AAST
    aast_xml = 'http://stistko.uiv.cz/katalog/textdata/C213436AAST.xml'
    aast = pd.read_xml(aast_xml, encoding='cp1250', xpath='./veta')
    aast['ISO_ALPHA3'] = aast['SPC'].str[2:5]
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STAT', 'ZKR': 'STAT_LABEL', 'ISO_ALPHA3': 'STAT_ISO'}), 
              how='left')
    df = pd.merge(df, aast[['KOD', 'ZKR', 'ISO_ALPHA3']].rename(columns={'KOD': 'STATB', 'ZKR': 'STATB_LABEL', 'ISO_ALPHA3': 'STATB_ISO'}), 
              how='left')
    
    # label OBSS - I am not including full school info
    #   -> actually, this give me slightly better match - probably because I am including also specializations that are not valid anymore
    li = []
    for y in range(1999, 2023):
        pdf = pd.read_csv(f'{data_root}/uchazec/stredni-skoly/m{y}.csv', encoding='cp1250')
        li.append(pdf)
    ss = pd.concat(li, axis=0, ignore_index=True).drop_duplicates().reset_index(drop=True)
    
    def selector(frame):
        # just use the longest one - the shorter ones typically use abbreviations
        frame['len'] = frame['PLN_NAZ'].str.strip().str.len()
        return frame.sort_values('len', ascending=False)['PLN_NAZ'].iloc[0]

    ss['OBOR'] = ss['OBOR'].str.strip()
    obory = ss[['OBOR', 'PLN_NAZ']].drop_duplicates()
    obory_uni = obory.groupby('OBOR')[['PLN_NAZ']].apply(selector).reset_index()
    df = pd.merge(df, obory_uni.rename(columns={'OBOR': 'OBSS', 0: 'OBSS_LABEL'}), how='left')
    
    sss = ss[['IZO', 'ZAR_PLN', 'VUSC']].drop_duplicates()
    df = pd.merge(df, sss.rename(columns={'IZO': 'IZOS', 'ZAR_PLN': 'IZOS_LABEL', 'VUSC': 'IZOS_NUTS'}), how='left')
    
    registers = {
        'ODHL': 'http://stistko.uiv.cz/katalog/textdata/C213729MCPP.xml',
        'TYP_ST': 'http://stistko.uiv.cz/katalog/textdata/C214019PASP.xml',
        'FORMA_ST' : 'http://stistko.uiv.cz/katalog/textdata/C214048PAFS.xml',
        'VYPR': 'http://stistko.uiv.cz/katalog/textdata/C214147MCPR.xml',
        'ZAPS': 'http://stistko.uiv.cz/katalog/textdata/C21427MCZR.xml',
        # 'OBSS': 'http://stistko.uiv.cz/katalog/textdata/C113922AKSO.xml'
    }
    
    for c, url in registers.items():
        rg = pd.read_xml(url, encoding='cp1250', xpath='./veta')
        df = pd.merge(df, rg[['KOD', 'TXT']].rename(columns={'KOD': c, 'TXT': f'{c}_LABEL'}), how='left')

    regpro = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/regpro.csv', encoding='cp1250')
    regpro['RID'] = regpro['RID'].str.strip()
    regpro_cols = ['ZAR_PLN', 'VS_PLN', 'ZAR_NAZ', 'UZEMI']
    regpro = regpro[['RID'] + regpro_cols].rename(columns={x: f'RID_{x}' for x in regpro_cols})
    df = pd.merge(df, regpro.rename(columns={'RID_UZEMI': 'RID_NUTS'}), how='left')
    
    program = pd.read_csv(f'{data_root}/uchazec/uch3-03-6/cisel/program.csv', encoding='cp1250')
    df = pd.merge(df, program.rename(columns={'KOD': 'PROGRAM', 'NAZEV': 'PROGRAM_LABEL'}), how='left')
    
    # plné obory
    akvo_xml = 'http://stistko.uiv.cz/katalog/textdata/C214117AKVO.xml'
    akvo = pd.read_xml(akvo_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 6):
        df = pd.merge(df, akvo[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR{i}', 'TXT': f'OBOR{i}_LABEL'}), how='left')
        
    # aprobace
    aaap_xml = 'http://stistko.uiv.cz/katalog/textdata/C214234AAAP.xml'
    aaap = pd.read_xml(aaap_xml, encoding='cp1250', xpath='./veta')
    for i in range(1, 4):
        df = pd.merge(df, aaap[['KOD', 'TXT']].rename(columns={'KOD': f'APRO{i}', 'TXT': f'APRO{i}_LABEL'}), how='left')
        
    # studijní obory podle skupin
    akko_xml = 'http://stistko.uiv.cz/katalog/textdata/C11240AKKO.xml'
    akko = pd.read_xml(akko_xml, encoding='cp1250', xpath='./veta')
    for i in [1, 2, 5]:
        df[f'OBOR1_{i}'] = df['OBOR1'].str[:i]
        df = pd.merge(df, akko[['KOD', 'TXT']].rename(columns={'KOD': f'OBOR1_{i}', 'TXT': f'OBOR1_{i}_LABEL'}), how='left')
    
    df['TYP_SS'] = np.where(df['OBSS'] == '', 'JINE', np.where(df['OBSS'].str.startswith('794'), 'G', 'SOS'))
    df['TYP_G'] = np.where(df['TYP_SS'] != 'G', np.nan, df['OBSS'].str[5])
    
    # fillna in some columns
    for c in ['PROGRAM_LABEL'] + [f'OBOR{i}_LABEL' for i in range(1, 6)] + [f'APRO{i}_LABEL' for i in range(1, 4)]:
        df[c] = df[c].fillna('')
    
    return df